In [ ]:
%load_ext autoreload
%autoreload 2
%aimport -sympy

In [ ]:
from src.planttraits.datasets.plant_traits_dataset import PlantTraitsDataset
from src.planttraits.models.example_model import PTNN, kwargs
from planttraits.utils import TARGET_COLUMN_NAMES
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
import torch 
import matplotlib.pyplot as plt
import os 
from pathlib import Path 
import pandas as pd

In [ ]:
train_dataset = PlantTraitsDataset()
test_dataset = PlantTraitsDataset(preprocessors=train_dataset.return_preprocessors())

### Modis Vod

In [ ]:
preps = train_dataset.return_preprocessors()
modis_prep = preps['modis_vod']
cols = modis_prep.columns
train_modis = train_dataset.data[cols]
display(train_modis.head())

In [ ]:
print(train_modis.columns.tolist())

In [ ]:
test_col = f"MODIS_2000.2020_monthly_mean_surface_reflectance_band_14_._month_m10"

if test_col in train_modis.columns:
    print("Istnieje")
else:
    print("Nie istnieje")


### Soil

In [ ]:
preps = train_dataset.return_preprocessors()
soil_prep = preps['soil']
cols = soil_prep.columns

In [ ]:
cols

In [ ]:
len(cols)

### Worldclim Bio

In [ ]:
preps = train_dataset.return_preprocessors()
worldclim_prep = preps['worldclimbio']
cols = worldclim_prep.selected_columns

In [ ]:
cols

### Images

In [ ]:
train_img = train_dataset.img_preprocessor.transform(r"C:\Users\julia\VSCode\plant-traits-2024\data\train_images\26375.jpeg")
train_img.shape

In [ ]:
plt.imshow(train_img.numpy().transpose(1, 2, 0))

### All Dataset

In [ ]:
len(train_dataset)

In [ ]:
len(test_dataset)

In [ ]:
train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)

In [ ]:
img, modisvod_row, soil_row, worldclimbio_row, std_row, mean_row = next(iter(train_loader))
# img, modisvod_row, soil_row, worldclimbio_row, std_row, mean_row = next(iter(test_loader))

In [ ]:
img.shape

In [ ]:
img

In [ ]:
modisvod_row

In [ ]:
soil_row

In [ ]:
worldclimbio_row

In [ ]:
std_row, mean_row

In [ ]:
torch.concat([modisvod_row, soil_row, worldclimbio_row], axis=1).shape

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
kwargs

In [ ]:
import torchvision

In [ ]:
efficientnet_b2 = torchvision.models.efficientnet_b2(weights='IMAGENET1K_V1')

In [ ]:
efficientnet_b2

In [ ]:
efficientnet_b2.classifier = torch.nn.Identity()

In [ ]:
efficientnet_b2(torch.rand((2, 3, 256, 256))).shape

In [ ]:
resnet18 = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)

In [ ]:
resnet18.fc = torch.nn.Identity() # żeby pozbyć się liniowego klasyfikatora i tylko wybrać reprezentację obrazka

In [ ]:
kwargs['backbone_net'] = resnet18

In [ ]:
kwargs

In [ ]:
checkpoint_dir = Path(f'../models')
os.makedirs(checkpoint_dir, exist_ok=True)
model_name = 'test-model'
logger = TensorBoardLogger(save_dir='./tnsr-logs', name=model_name)
checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath=checkpoint_dir / model_name, filename='{epoch}-{val_loss:.2f}-{train_loss:.2f}', 
                                                                                   monitor='train_r2_score', save_last=True) 
# early_stopping = pl.callbacks.EarlyStopping(monitor='val_loss', verbose=True)

model = PTNN(**kwargs)
trainer = pl.Trainer(max_epochs=3, fast_dev_run=False, default_root_dir=checkpoint_dir / model_name, logger=logger)

In [ ]:
trainer.fit(model, train_dataloaders=train_loader)

In [ ]:
test_preds = trainer.predict(model, test_loader)

In [ ]:
test_preds

In [ ]:
test_preds

In [ ]:
test_dataset.save_submission(test_preds, './submis.csv')